In [162]:
# IMPORTANTO BIBLIOTECAS A SEREM TRABALHADAS

import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
from plyer import notification 
import requests
from tqdm import tqdm

In [163]:
def alerta(nivel, base, etapa, erro=""):
    '''
        Alerta de falha de carregamento de base de dados
    '''
    now = str(datetime.now())

    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}\n{erro}"


    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel  == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nivel",nivel,"não disponível!")

    notification.notify(
            title=title,
            message=msg,
            app_name='alerta',
            timeout=10
        )


In [164]:
def tabelas_bd():
    '''
        Retorna um dataframe com as tabelas do banco de dados.
    '''
    conn = sqlite3.connect('pokemon.db')

    # Executar uma consulta que retorna as informações do esquema do banco de dados
    query = "SELECT name FROM sqlite_master WHERE type='table'"
    schema = pd.read_sql_query(query, conn)

    conn.close()

    return schema

def salva_bd(df, nome_tabela):
    '''
        Salva dataframe df na tabela nome_tabela.
    '''
    conn = sqlite3.connect('pokemon.db')

    # Escrever o DataFrame na tabela 'nome_tabela'
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)

    conn.close()

    return True

def carrega_bd(nome_tabela):
    '''
        Carrega tabela nome_tabela num dataframe. 
    '''
    conn = sqlite3.connect('pokemon.db')

    # Executar uma consulta SELECT na tabela 'produtos' e converter em um DataFrame
    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query, conn)

    conn.close()

    return df  

In [165]:
def get_json_api(url):
    '''
        Request GET url e retorna json de saida
    '''

    response = requests.get(url)

    if response.status_code == 200:
        data_json = response.json()
        return data_json
    
    # Erro
    raise Exception(f"erro request,  {response.status_code} - {url}\n{response.text}")

In [166]:
def get_base_pokemons_url():
    '''
        request completo iterativo da tabela pokemons_url
        retorna dataframe da tabela 
    '''
    next_url = "https://pokeapi.co/api/v2/pokemon"
    df_pokemons_full = pd.DataFrame()
    while next_url is not None:
        print(next_url, end='\r')
        
        # GET 
        json_data = get_json_api(next_url)

        #para dataframe
        df_pokemons = pd.DataFrame(json_data['results'])

        #append/concat  
        df_pokemons_full = pd.concat([df_pokemons_full,df_pokemons],ignore_index=True)

        #next page
        next_url = json_data['next']
    
    return df_pokemons_full

In [167]:
def get_base_habilidades_url():
    '''
        request completo iterativo da tabela habilidades_url
        retorna dataframe da tabela 
    '''
    next_url = "https://pokeapi.co/api/v2/ability"
    df_habilidades_full = pd.DataFrame()
    while next_url is not None:
        print(next_url, end='\r')
        
        # GET 
        json_data = get_json_api(next_url)

        #para dataframe
        df_habilidade = pd.DataFrame(json_data['results'])

        #append/concat  
        df_habilidades_full = pd.concat([df_habilidades_full,df_habilidade],ignore_index=True)

        #next page
        next_url = json_data['next']
    
    return df_habilidades_full

In [168]:
def get_base_especies_url():
    '''
        request completo iterativo da tabela especie_url
        retorna dataframe da tabela 
    '''
    next_url = "https://pokeapi.co/api/v2/pokemon-species"
    df_especie_full = pd.DataFrame()
    while next_url is not None:
        print(next_url, end='\r')
        
        # GET 
        json_data = get_json_api(next_url)

        #para dataframe
        df_especie = pd.DataFrame(json_data['results'])

        #append/concat  
        df_especie_full = pd.concat([df_especie_full,df_especie],ignore_index=True)

        #next page
        next_url = json_data['next']
    
    return df_especie_full

In [169]:
def get_base_pokemons():

    df_pokemons_url = carrega_bd("pokemons_url")

    df_pokemons_full = pd.DataFrame()

    for url in tqdm(df_pokemons_url['url'].values):

        #GET
        json_data = get_json_api(url)

        #para dataframe e append
        id              = json_data['id']
        name            = json_data['name']
        abilities       = json_data['abilities']
        height          = json_data['height']
        weight          = json_data['weight']
        base_experience = json_data['base_experience']
        species         = json_data['species']['url']

        sr_pokemon = pd.Series({
            "url": url,
            "pokemon_id": id,
            "name": name,
            "height": height,
            "weight": weight,
            "base_experience": base_experience,
            "habilidades": str(abilities),
            "especies": species
        })

        #append/concat 
        df_pokemons_full = pd.concat([df_pokemons_full,sr_pokemon.to_frame().T],ignore_index=True)
        
    return df_pokemons_full

In [170]:
def get_base_habilidades():

    df_habilidades_url = carrega_bd("habilidades_url")

    df_habilidades_full = pd.DataFrame()

    for url in tqdm(df_habilidades_url['url'].values):

        #GET
        json_data = get_json_api(url)

        #para dataframe e append
        id             = json_data['id']
        name           = json_data['name']
        generation     = json_data['generation']['name']
        is_main_series = json_data['is_main_series']
        effect         = "\n---\n".join([effect['effect'] for effect in json_data['effect_entries']])

        sr_habilidade = pd.Series({
            "url": url,
            "habilidade_id":id,
            "name": name,
            "generation": generation,
            "is_main_series": is_main_series,
            "effect": effect
        })

        #append/concat 
        df_habilidades_full = pd.concat([df_habilidades_full,sr_habilidade.to_frame().T],ignore_index=True)

    return df_habilidades_full

In [171]:
def get_base_especies():

    df_especie_url = carrega_bd("especie_url")

    df_especie_full = pd.DataFrame()

    for url in tqdm(df_especie_url['url'].values):

        #GET
        json_data = get_json_api(url)

        #para dataframe e append
        id             = json_data['id']
        name           = json_data['name']
        color          = json_data['color']['name']
        growth_rate    = json_data['growth_rate']['name']                

        sr_especie = pd.Series({
            "url": url,
            "especie_id":id,
            "name": name,
            "color": color,
            "growth_rate": growth_rate,                    
        })

        #append/concat 
        df_especie_full = pd.concat([df_especie_full,sr_especie.to_frame().T],ignore_index=True)

    return df_especie_full

In [172]:
def etapa_extracao():
    
    #################################################
    print("  > Base pokemons_url:\n")
    try:
        df_pokemons_url = get_base_pokemons_url()
        salva_bd(df_pokemons_url,"pokemons_url")
    except Exception as e:
        alerta(nivel = 3, 
               base = "pokemons_url", 
               etapa = "EXTRACAO", 
               erro=e)
    
    #################################################
    print("  > Base habilidades_url:\n")
    try:
        df_habilidades_url = get_base_habilidades_url()
        salva_bd(df_habilidades_url,"habilidades_url")
    except Exception as e:
        alerta(nivel = 3, 
               base = "habilidades_url", 
               etapa = "EXTRACAO", 
               erro=e)
        
    #################################################
    print("  > Base especie_url:\n")
    try:
        df_especie_url = get_base_especies_url()
        salva_bd(df_especie_url,"especie_url")
    except Exception as e:
        alerta(nivel = 3, 
               base = "especie_url", 
               etapa = "EXTRACAO", 
               erro=e)
        
    #################################################
    print("  > Base pokemons:\n")
    try:
        df_pokemons_full = get_base_pokemons()
        salva_bd(df_pokemons_full,"stage_pokemons")
    except Exception as e:
        alerta(nivel = 3, 
               base = "stage_pokemons", 
               etapa = "EXTRACAO", 
               erro=e)

    
    #################################################
    print("  > Base habilidades:\n")
    try:
        df_habilidades_full = get_base_habilidades()
        salva_bd(df_habilidades_full,"stage_habilidades")
    except Exception as e:
        alerta(nivel = 3, 
               base = "stage_habilidades", 
               etapa = "EXTRACAO", 
               erro=e)
        
    #################################################
    print("  > Base especies:\n")
    try:
        df_especie_full = get_base_especies()
        salva_bd(df_especie_full,"stage_especies")
    except Exception as e:
        alerta(nivel = 3, 
               base = "stage_especies", 
               etapa = "EXTRACAO", 
               erro=e)

In [173]:
""" def etapa_transformacao():

    try:
        #base de pokemons
        df_pokemons = carrega_bd("pokemons")        
        #abrindo as linhas por habilidade
        df_pokemons['habilidades'] = df_pokemons['habilidades'].apply(eval)
        df_pokemons_open = df_pokemons.explode('habilidades')
        #buscando id das habilidades
        df_pokemons_open['ability_url'] = df_pokemons_open['habilidades'].apply(lambda row: row['ability']['url'])
        df_pokemons_open['ability_id']  = df_pokemons_open['ability_url'].apply(lambda row: int(row.split('/')[-2]))
        df_pokemons_open = df_pokemons_open.rename(columns={'name':'pokemon_name'})
        df_pokemons_open = df_pokemons_open[['pokemon_id','ability_id','pokemon_name','height','weight','base_experience']]
        df_pokemons_tratado = df_pokemons_open.fillna({'base_experience':0})

        #base de habilidades
        df_habilidades = carrega_bd("habilidades")
        df_habilidades = df_habilidades.rename(columns={'name':'ability_name','effect':'ability_effect'})
        df_habilidades = df_habilidades[['ability_id','ability_name','ability_effect']]

        #base de especies
        df_especies = carrega_bd("especies")
        df_especies = df_especies.rename(columns={'specie_id':'id_specie','name':'pokemon_name'})
        df_especies = df_especies[['id_specie','pokemon_name','color','growth_rate']]        

        #join da base pokemons e habilidades
        df_pokemons_completo = df_pokemons_tratado.merge(df_habilidades,on=['ability_id'],how='left').merge(df_especies,on=['pokemon_name'],how='left')

        #salva resultado final
        salva_bd(df_pokemons_completo,"pokemons_completo")
        print("pokemons_completo salva")
    except Exception as e:
        alerta(nivel = 2, 
                base = "pokemons_completo", 
                etapa = "TRANSFORMACAO", 
                erro=e) """

' def etapa_transformacao():\n\n    try:\n        #base de pokemons\n        df_pokemons = carrega_bd("pokemons")        \n        #abrindo as linhas por habilidade\n        df_pokemons[\'habilidades\'] = df_pokemons[\'habilidades\'].apply(eval)\n        df_pokemons_open = df_pokemons.explode(\'habilidades\')\n        #buscando id das habilidades\n        df_pokemons_open[\'ability_url\'] = df_pokemons_open[\'habilidades\'].apply(lambda row: row[\'ability\'][\'url\'])\n        df_pokemons_open[\'ability_id\']  = df_pokemons_open[\'ability_url\'].apply(lambda row: int(row.split(\'/\')[-2]))\n        df_pokemons_open = df_pokemons_open.rename(columns={\'name\':\'pokemon_name\'})\n        df_pokemons_open = df_pokemons_open[[\'pokemon_id\',\'ability_id\',\'pokemon_name\',\'height\',\'weight\',\'base_experience\']]\n        df_pokemons_tratado = df_pokemons_open.fillna({\'base_experience\':0})\n\n        #base de habilidades\n        df_habilidades = carrega_bd("habilidades")\n        df_h

In [174]:
def tranformacao_pokemon():

    try:
        #base de pokemons
        df_pokemons = carrega_bd("stage_pokemons") 
        
        #tratando missing values pokemons
        df_pokemons_tratado = df_pokemons.fillna({'base_experience':0})

        #tratando tipos de campo pokemons
        df_pokemons_tratado['pokemon_id'] = df_pokemons_tratado['pokemon_id'].astype(int)
        df_pokemons_tratado['name'] = df_pokemons_tratado['name'].astype(str)
        df_pokemons_tratado['height'] = df_pokemons_tratado['height'].astype(int)
        df_pokemons_tratado['weight'] = df_pokemons_tratado['weight'].astype(int)
        df_pokemons_tratado['base_experience'] = df_pokemons_tratado['base_experience'].astype(float)
        df_pokemons_tratado = df_pokemons_tratado[['pokemon_id','name','height','weight','base_experience']]

        #salva tabela de pokemons e habilidades
        salva_bd(df_pokemons_tratado,"pokemons")
        print("pokemons salva")
    except Exception as e:
        alerta(nivel = 2, 
                base = "pokemons", 
                etapa = "TRANSFORMACAO", 
                erro=e)

In [175]:
def tranformacao_habilidades():

    try:
        #base de habilidades
        df_habilidades = carrega_bd("stage_habilidades")
        df_habilidades = df_habilidades.rename(columns={'name':'ability_name','effect':'ability_effect'})
        df_habilidades = df_habilidades[['habilidade_id','ability_name','ability_effect']]

        #tratando tipos de campo habilidades
        df_habilidades['habilidade_id'] = df_habilidades['habilidade_id'].astype(int)
        df_habilidades['ability_name'] = df_habilidades['ability_name'].astype(str)
        df_habilidades['ability_effect'] = df_habilidades['ability_effect'].astype(str)

        #salva tabela de pokemons e habilidades
        salva_bd(df_habilidades,"habilidades")
        print("habilidades salva")
    except Exception as e:
        alerta(nivel = 2, 
                base = "habilidades", 
                etapa = "TRANSFORMACAO", 
                erro=e)

In [176]:
def tranformacao_pokemons_habilidades():

    try:
        #base de pokemons
        df_pokemons = carrega_bd("stage_pokemons") 

        #abrindo as linhas por habilidade
        df_pokemons['habilidades'] = df_pokemons['habilidades'].apply(eval)
        df_pokemons_open = df_pokemons.explode('habilidades')

        #buscando id das habilidades
        df_pokemons_open['ability_url'] = df_pokemons_open['habilidades'].apply(lambda row: row['ability']['url'])
        df_pokemons_open['habilidade_id']  = df_pokemons_open['ability_url'].apply(lambda row: int(row.split('/')[-2]))        
        df_pokemons_open = df_pokemons_open[['pokemon_id','habilidade_id']]

        #removendo vazios
        df_poke_hab_vazios = df_pokemons_open.dropna()

        #removendo duplicadas
        df_poke_hab_tratado = df_poke_hab_vazios.drop_duplicates()

        #tratando tipos de campos
        df_poke_hab_tratado['pokemon_id'] = df_poke_hab_tratado['pokemon_id'].astype(int)
        df_poke_hab_tratado['habilidade_id'] = df_poke_hab_tratado['habilidade_id'].astype(int)

        #salva tabela de pokemons e habilidades
        salva_bd(df_poke_hab_tratado,"pokemons_habilidades")
        print("pokemons_habilidades salva")
    except Exception as e:
        alerta(nivel = 2, 
                base = "pokemons_habilidades", 
                etapa = "TRANSFORMACAO", 
                erro=e)

In [177]:
def tranformacao_especies():

    try:
        #base de especies
        df_especies = carrega_bd("stage_especies")        
        df_especies = df_especies[['especie_id','pokemon_name','color','growth_rate']] 

        #removendo vazios
        df_especies_tratado = df_especies.dropna()

        #tratando tipos de campos
        df_especies_tratado['especie_id'] = df_especies_tratado['especie_id'].astype(int)
        df_especies_tratado['pokemon_name'] = df_especies_tratado['pokemon_name'].astype(str)
        df_especies_tratado['color'] = df_especies_tratado['color'].astype(str)
        df_especies_tratado['growth_rate'] = df_especies_tratado['growth_rate'].astype(str)

        #salva tabela de pokemons e habilidades
        salva_bd(df_especies_tratado,"especies")
        print("especies salva")
    except Exception as e:
        alerta(nivel = 2, 
                base = "especies", 
                etapa = "TRANSFORMACAO", 
                erro=e)

In [178]:
def tranformacao_pokemons_especies():

    try:
        #base de pokemons
        df_pokemons = carrega_bd("stage_pokemons") 

        #buscando id das habilidades
        df_pokemons['especie_id']  = df_pokemons['especies'].apply(lambda row: int(row.split('/')[-2]))      
        df_pokemons = df_pokemons[['pokemon_id','especie_id']]

        #removendo vazios
        df_poke_spe_vazios = df_pokemons.dropna()

        #removendo duplicadas
        df_poke_spe_tratado = df_poke_spe_vazios.drop_duplicates()

        #tratando tipos de campos
        df_poke_spe_tratado['pokemon_id'] = df_poke_spe_tratado['pokemon_id'].astype(int)
        df_poke_spe_tratado['especie_id'] = df_poke_spe_tratado['especie_id'].astype(int)

        #salva tabela de pokemons e habilidades
        salva_bd(df_poke_spe_tratado,"pokemons_especies")
        print("pokemons_especies salva")
    except Exception as e:
        alerta(nivel = 2, 
                base = "pokemons_especies", 
                etapa = "TRANSFORMACAO", 
                erro=e)

In [179]:
etapa_extracao()

  > Base pokemons_url:

  > Base habilidades_url:/pokemon?offset=1300&limit=20

  > Base especie_url:i/v2/ability?offset=360&limit=70

  > Base pokemons:/api/v2/pokemon-species?offset=1020&limit=50



100%|██████████| 1302/1302 [10:48<00:00,  2.01it/s]


  > Base habilidades:



100%|██████████| 367/367 [03:34<00:00,  1.71it/s]


  > Base especies:



100%|██████████| 1025/1025 [08:37<00:00,  1.98it/s]


In [180]:
tranformacao_pokemon()

pokemons salva


In [181]:
tranformacao_habilidades()

habilidades salva


In [182]:
tranformacao_especies()

In [183]:
tranformacao_pokemons_habilidades()

C:\Users\Carlos.souza\AppData\Local\Temp\ipykernel_49760\1133406227.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_poke_hab_tratado['pokemon_id'] = df_poke_hab_tratado['pokemon_id'].astype(int)
C:\Users\Carlos.souza\AppData\Local\Temp\ipykernel_49760\1133406227.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_poke_hab_tratado['habilidade_id'] = df_poke_hab_tratado['habilidade_id'].astype(int)


pokemons_habilidades salva


In [184]:
tranformacao_pokemons_especies()

pokemons_especies salva


In [185]:
tabelas_bd()

,name
0,pokemons_completo
1,especies
2,pokemons_url
3,habilidades_url
4,especie_url
5,stage_pokemons
6,stage_habilidades
7,stage_especies
8,pokemons
9,habilidades


In [186]:
carrega_bd('pokemons_url')

,name,url
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/
2,venusaur,https://pokeapi.co/api/v2/pokemon/3/
3,charmander,https://pokeapi.co/api/v2/pokemon/4/
4,charmeleon,https://pokeapi.co/api/v2/pokemon/5/
...,...,...
1297,ogerpon-wellspring-mask,https://pokeapi.co/api/v2/pokemon/10273/
1298,ogerpon-hearthflame-mask,https://pokeapi.co/api/v2/pokemon/10274/
1299,ogerpon-cornerstone-mask,https://pokeapi.co/api/v2/pokemon/10275/
1300,terapagos-terastal,https://pokeapi.co/api/v2/pokemon/10276/


In [187]:
carrega_bd('habilidades_url')

,name,url
0,stench,https://pokeapi.co/api/v2/ability/1/
1,drizzle,https://pokeapi.co/api/v2/ability/2/
2,speed-boost,https://pokeapi.co/api/v2/ability/3/
3,battle-armor,https://pokeapi.co/api/v2/ability/4/
4,sturdy,https://pokeapi.co/api/v2/ability/5/
...,...,...
362,run-up,https://pokeapi.co/api/v2/ability/10056/
363,conqueror,https://pokeapi.co/api/v2/ability/10057/
364,shackle,https://pokeapi.co/api/v2/ability/10058/
365,decoy,https://pokeapi.co/api/v2/ability/10059/


In [188]:
carrega_bd('especie_url')

,name,url
0,bulbasaur,https://pokeapi.co/api/v2/pokemon-species/1/
1,ivysaur,https://pokeapi.co/api/v2/pokemon-species/2/
2,venusaur,https://pokeapi.co/api/v2/pokemon-species/3/
3,charmander,https://pokeapi.co/api/v2/pokemon-species/4/
4,charmeleon,https://pokeapi.co/api/v2/pokemon-species/5/
...,...,...
1020,raging-bolt,https://pokeapi.co/api/v2/pokemon-species/1021/
1021,iron-boulder,https://pokeapi.co/api/v2/pokemon-species/1022/
1022,iron-crown,https://pokeapi.co/api/v2/pokemon-species/1023/
1023,terapagos,https://pokeapi.co/api/v2/pokemon-species/1024/


In [189]:
carrega_bd('pokemons')

,pokemon_id,name,height,weight,base_experience
0,1,bulbasaur,7,69,64.0
1,2,ivysaur,10,130,142.0
2,3,venusaur,20,1000,263.0
3,4,charmander,6,85,62.0
4,5,charmeleon,11,190,142.0
...,...,...,...,...,...
1297,10273,ogerpon-wellspring-mask,12,398,0.0
1298,10274,ogerpon-hearthflame-mask,12,398,0.0
1299,10275,ogerpon-cornerstone-mask,12,398,0.0
1300,10276,terapagos-terastal,3,160,0.0


In [190]:
carrega_bd('habilidades')

,habilidade_id,ability_name,ability_effect
0,1,stench,Attacken die Schaden verursachen haben mit jed...
1,2,drizzle,Das weather ändert sich zu rain wenn ein Pokém...
2,3,speed-boost,Die speed des Pokémon erhöht sich jede Runde u...
3,4,battle-armor,Attacken können keinen kritischen Treffer land...
4,5,sturdy,"Wenn das Pokémon volle hp hat, lässt jeder Tre..."
...,...,...,...
362,10056,run-up,
363,10057,conqueror,
364,10058,shackle,
365,10059,decoy,


In [191]:
carrega_bd('especies')

,url,id,name,color,growth_rate
0,https://pokeapi.co/api/v2/pokemon-species/1/,1,bulbasaur,green,medium-slow
1,https://pokeapi.co/api/v2/pokemon-species/2/,2,ivysaur,green,medium-slow
2,https://pokeapi.co/api/v2/pokemon-species/3/,3,venusaur,green,medium-slow
3,https://pokeapi.co/api/v2/pokemon-species/4/,4,charmander,red,medium-slow
4,https://pokeapi.co/api/v2/pokemon-species/5/,5,charmeleon,red,medium-slow
...,...,...,...,...,...
1020,https://pokeapi.co/api/v2/pokemon-species/1021/,1021,raging-bolt,yellow,slow
1021,https://pokeapi.co/api/v2/pokemon-species/1022/,1022,iron-boulder,gray,slow
1022,https://pokeapi.co/api/v2/pokemon-species/1023/,1023,iron-crown,blue,slow
1023,https://pokeapi.co/api/v2/pokemon-species/1024/,1024,terapagos,blue,slow


In [192]:
carrega_bd('pokemons_habilidades')


,pokemon_id,habilidade_id
0,1,65
1,1,34
2,2,65
3,2,34
4,3,65
...,...,...
2898,10273,11
2899,10274,104
2900,10275,5
2901,10276,305


In [193]:
carrega_bd('pokemons_especies')

,pokemon_id,especie_id
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
1297,10273,1017
1298,10274,1017
1299,10275,1017
1300,10276,1024
